<a href="https://colab.research.google.com/github/vcorredorg/SenalesySistemas2025/blob/main/Taller2/Transformada_de_%20Fourier/App_streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#APP comunicaciones- modulación AM

In [ ]:
#instalación de librerías
!pip install streamlit -q


In [ ]:
!mkdir pages

mkdir: cannot create directory ‘pages’: File exists


In [ ]:
%%writefile modulacion_AM.py

import streamlit as st
st.set_page_config(
    page_title="Comunicaciones AM", page_icon="📡", layout="wide"
)

st.write("# Bienvenido! 👋")

st.sidebar.success("Seleccciona una demo a explorar.")

st.markdown(
    """
    Desde Streamlit se explorarán los conceptos de comunicaciones y modulación AM **👈 Seleccione una opción de la barra lateral** para ver el demo correspondiente.
"""
)

Overwriting modulacion_AM.py


In [ ]:

%%writefile 1_📈_Espectro_en_frecuencia.py

import streamlit as st
import time
import numpy as np

st.set_page_config(page_title="Espectro en frecuencia", page_icon="📈")

st.markdown("# Espectro en frecuencia")
st.sidebar.header("Espectro en frecuencia")

st.write("Sea la señal portadora c(t) = Ac cos(2πFct), con Ac, Fc ∈ ℝ, y la señal mensaje m(t) ∈ ℝ.")
st.write("Encuentre el espectro en frecuencia de la señal modulada en amplitud (AM):")

st.latex(r"""
y(t) = \frac{1 + m(t)}{A_c} \, c(t)
""")
st.write("y(t) = (1 + A_c m(t)) c(t) = A_c cos(2πF_c t) + m(t) cos(2πF_c t).")

st.write("Propiedades de Transformada de Fourier que usaremos:")

st.latex(r"""
\mathcal{F}\{\cos(2\pi F_c t)\}
= \tfrac12\big[\delta(f-F_c)+\delta(f+F_c)\big]
""")

st.latex(r"""
\mathcal{F}\{x(t)\cos(2\pi F_c t)\}
= \tfrac12\big[X(f-F_c)+X(f+F_c)\big]
""")

st.write("Denotemos M(f) = 𝓕{m(t)}.")

st.write("Primer término:")

st.latex(r"""
\mathcal{F}\{A_c\cos(2\pi F_c t)\}
= \frac{A_c}{2}\big[\delta(f-F_c)+\delta(f+F_c)\big]
""")

st.write("Segundo término:")

st.latex(r"""
\mathcal{F}\{m(t)\cos(2\pi F_c t)\}
= \tfrac12\big[M(f-F_c)+M(f+F_c)\big]
""")

st.write("Espectro de la AM con portadora:")

st.latex(r"""
Y(f)=\frac{A_c}{2}\big[\delta(f-F_c)+\delta(f+F_c)\big]
+\frac12\big[M(f-F_c)+M(f+F_c)\big]
""")

st.write("Interpretación:")
st.write("""
- Líneas de portadora en ±F_c con amplitud A_c/2.
- Bandas laterales: copias de M(f) desplazadas a ±F_c y escaladas por 1/2.
- Si M(f) está acotado a |f| ≤ B, entonces Y(f) ocupa [F_c−B, F_c+B] y [−F_c−B, −F_c+B].
- Para evitar sobre–modulación, típicamente |m(t)| ≤ A_c.
""")

Writing 1_📈_Espectro_en_frecuencia.py


In [ ]:
!mv  1_📈_Espectro_en_frecuencia.py pages/

In [ ]:
%%writefile 2_🌍_Mapping_Demo.py

import streamlit as st
import pandas as pd
import pydeck as pdk
from urllib.error import URLError

st.set_page_config(page_title="Mapping Demo", page_icon="🌍")

st.markdown("# Mapping Demo")
st.sidebar.header("Mapping Demo")
st.write(
    """Esta demo muestra como usar
[`st.pydeck_chart`](https://docs.streamlit.io/develop/api-reference/charts/st.pydeck_chart)
para visualizar información geoespacial."""
)


@st.cache_data
def from_data_file(filename):
    url = (
        "http://raw.githubusercontent.com/streamlit/"
        "example-data/master/hello/v1/%s" % filename
    )
    return pd.read_json(url)


try:
    ALL_LAYERS = {
        "Bike Rentals": pdk.Layer(
            "HexagonLayer",
            data=from_data_file("bike_rental_stats.json"),
            get_position=["lon", "lat"],
            radius=200,
            elevation_scale=4,
            elevation_range=[0, 1000],
            extruded=True,
        ),
        "Bart Stop Exits": pdk.Layer(
            "ScatterplotLayer",
            data=from_data_file("bart_stop_stats.json"),
            get_position=["lon", "lat"],
            get_color=[200, 30, 0, 160],
            get_radius="[exits]",
            radius_scale=0.05,
        ),
        "Bart Stop Names": pdk.Layer(
            "TextLayer",
            data=from_data_file("bart_stop_stats.json"),
            get_position=["lon", "lat"],
            get_text="name",
            get_color=[0, 0, 0, 200],
            get_size=15,
            get_alignment_baseline="'bottom'",
        ),
        "Outbound Flow": pdk.Layer(
            "ArcLayer",
            data=from_data_file("bart_path_stats.json"),
            get_source_position=["lon", "lat"],
            get_target_position=["lon2", "lat2"],
            get_source_color=[200, 30, 0, 160],
            get_target_color=[200, 30, 0, 160],
            auto_highlight=True,
            width_scale=0.0001,
            get_width="outbound",
            width_min_pixels=3,
            width_max_pixels=30,
        ),
    }
    st.sidebar.markdown("### Capas de mapa")
    selected_layers = [
        layer
        for layer_name, layer in ALL_LAYERS.items()
        if st.sidebar.checkbox(layer_name, True)
    ]
    if selected_layers:
        st.pydeck_chart(
            pdk.Deck(
                map_style="mapbox://styles/mapbox/light-v9",
                initial_view_state={
                    "latitude": 37.76,
                    "longitude": -122.4,
                    "zoom": 11,
                    "pitch": 50,
                },
                layers=selected_layers,
            )
        )
    else:
        st.error("Por favor elija al menos una capa arriba.")
except URLError as e:
    st.error(
        """
        **Esta demo requiere conexión a internet.**
        Connection error: %s
    """
        % e.reason
    )

Writing 2_🌍_Mapping_Demo.py


In [ ]:
!mv 2_🌍_Mapping_Demo.py pages/

In [ ]:

%%writefile 3_📊_DataFrame_Demo.py

import streamlit as st
import pandas as pd
import altair as alt
from urllib.error import URLError

st.set_page_config(page_title="DataFrame Demo", page_icon="📊")

st.markdown("# DataFrame Demo")
st.sidebar.header("DataFrame Demo")
st.write(
    """Esta demo muestra como usar `st.write` para visualizar Pandas DataFrames.
(Datos cortesia de [UN Data Explorer](http://data.un.org/Explorer.aspx).)"""
)


@st.cache_data
def get_UN_data():
    AWS_BUCKET_URL = "http://streamlit-demo-data.s3-us-west-2.amazonaws.com"
    df = pd.read_csv(AWS_BUCKET_URL + "/agri.csv.gz")
    return df.set_index("Region")


try:
    df = get_UN_data()
    countries = st.multiselect(
        "Seleccione los paises", list(df.index), ["China", "United States of America"]
    )
    if not countries:
        st.error("Por favor seleccione al menos un pais.")
    else:
        data = df.loc[countries]
        data /= 1000000.0
        st.write("### Producción Agricola Bruta ($B)", data.sort_index())

        data = data.T.reset_index()
        data = pd.melt(data, id_vars=["index"]).rename(
            columns={"index": "year", "value": "Gross Agricultural Product ($B)"}
        )
        chart = (
            alt.Chart(data)
            .mark_area(opacity=0.3)
            .encode(
                x="year:T",
                y=alt.Y("Gross Agricultural Product ($B):Q", stack=None),
                color="Region:N",
            )
        )
        st.altair_chart(chart, use_container_width=True)
except URLError as e:
    st.error(
        """
        **Esta demo requiere conexión a internet.**
        Connection error: %s
    """
        % e.reason
    )

Writing 3_📊_DataFrame_Demo.py


In [ ]:

!mv 3_📊_DataFrame_Demo.py pages/

In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 /usr/local/bin/cloudflared

#Ejecutar Streamlit
!streamlit run modulacion_AM.py &>/content/logs.txt &

#Exponer el puerto 8501 con Cloudflare Tunnel
!cloudflared tunnel --url http://localhost:8501 > /content/cloudflared.log 2>&1 &

#Leer la URL pública generada por Cloudflare
import time
time.sleep(5)  # Esperar que se genere la URL

import re
found_context = False  # Indicador para saber si estamos en la sección correcta

with open('/content/cloudflared.log') as f:
    for line in f:
        #Detecta el inicio del contexto que nos interesa
        if "Your quick Tunnel has been created" in line:
            found_context = True

        #Busca una URL si ya se encontró el contexto relevante
        if found_context:
            match = re.search(r'https?://\S+', line)
            if match:
                url = match.group(0)  #Extrae la URL encontrada
                print(f'Tu aplicación está disponible en: {url}')
                break  #Termina el bucle después de encontrar la URL

--2025-11-10 04:40:00--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64 [following]
--2025-11-10 04:40:00--  https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/955e9d1b-ac5e-4188-8867-e5f53958a8fe?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-11-10T05%3A31%3A59Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-11-10

In [ ]:
import os

res = input("Digite (1) para finalizar la ejecución del Dashboard: ")

if res.upper() == "1":
    os.system("pkill streamlit")  # Termina el proceso de Streamlit
    print("El proceso de Streamlit ha sido finalizado.")


Digite (1) para finalizar la ejecución del Dashboard: 0
